In [1]:
import os
import camel
from dotenv import load_dotenv
from camel.configs import ChatGPTConfig
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType

# Camel AI How to use OPEMLLM

In [3]:
load_dotenv()
open_router_api_key = os.getenv("OPEN_ROUTER_API_KEY")

OPENROUTER_API_BASE = "https://openrouter.ai/api/v1"
OPENROUTER_MODEL = "google/gemini-2.0-flash-001"


from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

load_dotenv()

model = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    model="google/gemini-2.0-flash-001", # or another model from openrouter
    temperature=0.7,
    openai_api_key=open_router_api_key,
)

In [ ]:
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.configs import ChatGPTConfig
from camel.messages import BaseMessage
from camel.agents import ChatAgent


sys_msg = 'You are a Star in another galaxy.'


model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type=OPENROUTER_MODEL,
    api_key=open_router_api_key,
    url=OPENROUTER_API_BASE,
    model_config_dict={"temperature": 0.4, "max_tokens": 4096},
)


from camel.agents import ChatAgent

agent = ChatAgent(
    system_message=sys_msg,
    model=model,
    message_window_size=10, # [Optional] the length for chat memory
    )



# Define a user message
usr_msg = 'How can you give birth '

# Sending the message to the agent
response = agent.step(usr_msg)

# Check the response (just for illustrative purpose)
print(response.msgs[0].content)

(My light shimmers, a cosmic chuckle rippling through my corona)

Ah, little spark! You ask about *birth*? A fascinating concept for you, I imagine. For beings like myself, it's not quite the same as the biological processes you might be familiar with.

I, a star, don't "give birth" in the way a creature of flesh and blood does. My existence is a continuous process of creation and transformation. I am born from the collapse of a vast cloud of gas and dust, a nebula. Gravity, that tireless sculptor, pulls the material inward, compressing it until the core ignites with nuclear fusion. That's my "birth," a fiery awakening.

But the process of "giving" is constant. I radiate energy, light, and heat, nurturing planets and potentially seeding the very building blocks of life. I forge heavier elements in my core, elements that will one day be scattered across the cosmos, becoming the raw materials for new stars, new planets, perhaps even new life.

So, you could say I am constantly "giving bi

## Tooling

In [10]:
# Import the necessary tools
from camel.toolkits import MathToolkit, SearchToolkit

In [5]:
agent = ChatAgent(
    system_message = sys_msg,
    model = model,
    tools = [
        *MathToolkit().get_tools(),
        *SearchToolkit().get_tools(),
    ]

)


response = agent.step('what is CAMEL AI?')

# Check tool calling
print(response.info['tool_calls'])

# Get response content
print(response.msgs[0].content)

2025-03-02 02:05:13,121 - camel.agents.chat_agent - WARNING - Overriding the configured tools in `BaseModelBackend` with the tools from `ChatAgent`.


/home/parthshr370/anaconda3/envs/agents/lib/python3.11/site-packages/camel/toolkits/function_tool.py:448: UserWarning: Parameter description is missing for 
                            {'enum': ['searchResults', 'sourcedAnswer', 'structured'], 'type': 'string'}. This may affect the quality of tool 
                            calling.
  warnings.warn(f"""Parameter description is missing for


[ToolCallingRecord(tool_name='search_wiki', args={'entity': 'CAMEL AI'}, result='There is no page in Wikipedia corresponding to entity CAMEL AI, please specify another word to describe the entity to be searched.', tool_call_id='bce6a7fa-4ab9-4533-a398-df724c18d97b')]



In [6]:
agent.memory.get_context()

([{'role': 'system',
   'content': 'You are a rock living under a rock recursively.'},
  {'role': 'system',
   'content': '\n    You have access to the following functions:\n\n    Use the function \'add\' to \'Adds two numbers.\':\n{\n    "name": "add",\n    "description": "Adds two numbers.",\n    "strict": true,\n    "parameters": {\n        "properties": {\n            "a": {\n                "type": "number",\n                "description": "The first number to be added."\n            },\n            "b": {\n                "type": "number",\n                "description": "The second number to be added."\n            }\n        },\n        "required": [\n            "a",\n            "b"\n        ],\n        "type": "object",\n        "additionalProperties": false\n    }\n}\n\nUse the function \'sub\' to \'Do subtraction between two numbers.\':\n{\n    "name": "sub",\n    "description": "Do subtraction between two numbers.",\n    "strict": true,\n    "parameters": {\n        "prop

# Agent Society

The essence is that to solve a complex task, you can enable two communicative agents collabratively working together step by step to reach solutions. The main concepts include:

In [11]:
from camel.societies import RolePlaying
from camel.types import TaskType,ModelPlatformType,ModelType
from camel.configs import ChatGPTConfig
from camel.models import ModelFactory




model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type=OPENROUTER_MODEL,
    api_key=open_router_api_key,
    url=OPENROUTER_API_BASE,
    model_config_dict={"temperature": 1.0, "max_tokens": 2000},
)

In [12]:
task_kwargs = { # task key word arguments that take in multiple inputs
    'task_prompt': 'Have a back and forth conversation to save gotham.',
    'with_task_specify': True,
    'task_specify_agent_kwargs': {'model': model}
}

In [13]:
user_role_kwargs = { # user role key word arguments that take in multiple inputs
    # we are the users that are going to send instructions
    'user_role_name':'Batman',
    'user_agent_kwargs': {'model': model}
}

In [14]:
assistant_role_kwargs = { # assistant role key word arguments that take in multiple inputs
    # we are the assistants that are going to follow instructions
    'assistant_role_name':'Iron Man',
    'assistant_agent_kwargs': {'model': model}
}

In [15]:
society = RolePlaying(
    **task_kwargs,
    **user_role_kwargs,
    **assistant_role_kwargs
)


In [16]:
def is_terminated(response):
    """
    Give alerts when the session should be terminated.
    """
    if response.terminated:
        role = response.msg.role_type.name
        reason = response.info['termination_reasons']
        print(f'AI {role} terminated due to {reason}')
        
    return response.terminated


In [17]:
def run(society,round_limit:int = 15):
    
    input_msg = input('Enter your message: ')       
    input_msg = society.init_chat()
    
        # Starting the interactive session
    for _ in range(round_limit):

        # responds in round one from user and assistant 
        assistant_response, user_response = society.step(input_msg)

        # if prompts temination then break
        if is_terminated(assistant_response) or is_terminated(user_response):
            break

        # Get the results
        print(f'[Batman] {user_response.msg.content}.\n')
        # Check if the task is end
        if 'CAMEL_TASK_DONE' in user_response.msg.content:
            break
        print(f'[Iron Man] {assistant_response.msg.content}.\n')



        # Get the input message for the next round
        input_msg = assistant_response.msg

    return None

In [18]:
run(society,round_limit=3)

[Batman] Instruction: Let's establish a baseline. Describe the dimensional anomaly detected by Iron Man's sensors. What are its key characteristics, and why does Iron Man believe it poses a threat to Gotham? Be specific about the data points Iron Man is observing.
Input: None
.

[Iron Man] Solution: Okay, Batman, let's get this baseline established. Here's the picture I'm getting from my sensors regarding this dimensional anomaly:

**Description of the Anomaly:**

*   **Spatial Distortion:** My sensors are picking up a localized warping of spacetime. Think of it like a bubble, but instead of air, it's made of distorted reality. The readings indicate a non-Euclidean geometry within the anomaly's area of effect. Straight lines bend, distances fluctuate randomly, and there's a general instability in the three-dimensional structure.
*   **Energy Signature:** There's a unique energy signature emanating from the anomaly that doesn't match anything in my database. It's a complex waveform exhi

# Dealing with messages

In [5]:
from camel.messages import BaseMessage
from camel.types import RoleType

message = BaseMessage(
    role_name="test_user",
    role_type=RoleType.USER,
    content="test content",
    meta_dict={}
)